### CAPSTONE-2: AUTOMATE NEURAL STYLE TRANSFER FOR ANIMATION AND ARTISTIC DESIGN COMPANIES
#### DATA WRANGLING

In [51]:
# Import Modules and data from Data sources: https://osf.io/jum2f/ AND https://osf.io/BTWNQ/
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import random as rd
# from dotenv import load_dotenv
# import requests as r
import tensorflow as tf
import kaggle as kg
from kaggle.api.kaggle_api_extended import KaggleApi
k_api = KaggleApi()
k_api.authenticate()

# load_dotenv(".env")
# API_KEY = os.getenv('OSF_ADMIN_KEY')
# print(API_KEY)

This dataset is really quite perfect. There really is only a couple of matters to address with it.  There is a scoring system here which I don't presume to understand but could be relevant to the NST project and so I will read the paper that came included with the dataset as it was used in a pure science project prior.  There are a number of columns I may have to drop due to their lack of relevance to the project but otherwise, as far as I could tell on a quick glance through every image is unique which is good, however there are number of NaN values which will need my attention.

In [38]:
# Load Metadata for images to be transformed by neural transfer
# params = {} 
# r.get(f"")
objects_df = pd.read_csv('.\Things_DB\THINGS\Things_concepts.tsv', sep='\t')
objects_df


,Word,uniqueID,Example image,Bigram,Percent_known,Rank (combining COCA/concreteness),Concreteness (M),COCA word freq (online),COCA word freq,COCA (dispersion),...,Top-down Category (WordNet),Top-down Category (manual selection),All Bottom-up Categories,WordNet Synonyms,WordNet ID,Wordnet ID2,Wordnet ID3,Wordnet ID4,"Definition (from WordNet, Google, or Wikipedia)",Number of word meanings in list
0,aardvark,aardvark,https://imgur.com/LAJGlN0,0,1.00,18158.5,4.68,53,28.0,0.78,...,animal,animal,animal,"aardvark, ant_bear, anteater, Orycteropus_afer",n02082791,aardvark%1:05:00::,aardvark#1,aardvark.n.01,nocturnal burrowing mammal of the grasslands o...,1
1,abacus,abacus,https://imgur.com/peZeM0l,0,0.93,17741.0,4.52,188,97.0,0.86,...,NaN,home decor,mathematical device,abacus,n02666196,abacus%1:06:00::,abacus#2,abacus.n.02,a calculator that performs arithmetic function...,1
2,accordion,accordion,https://imgur.com/GgGvdZR,0,0.97,8541.0,4.86,816,735.0,0.90,...,musical instrument,musical instrument,musical instrument,"accordion, piano_accordion, squeeze_box",n02672831,accordion%1:06:00::,accordion#1,accordion.n.01,a portable box-shaped free-reed instrument; th...,1
3,acorn,acorn,https://imgur.com/YfIB5lM,0,1.00,7723.5,4.96,1289,692.0,0.85,...,fruit,NaN,nut,acorn,n12267677,acorn%1:20:00::,acorn#1,acorn.n.01,fruit of the oak tree: a smooth thin-walled nu...,1
4,air conditioner,air_conditioner,https://imgur.com/KqYNwWH,1,1.00,12115.0,4.93,943,NaN,NaN,...,NaN,electronic device,home appliance,"air_conditioner, air_conditioning",n02686379,air_conditioner%1:06:00::,air_conditioner#1,air_conditioner.n.01,a system that keeps air cool and dry,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849,yoke,yoke,https://imgur.com/nOt3K3f,0,0.97,9558.5,4.83,692,597.0,0.90,...,NaN,NaN,"farm equipment, animal harness",yoke,n04612840,yoke%1:06:00::,yoke#7,yoke.n.07,stable gear that joins two draft animals at th...,1
1850,yolk,yolk,https://imgur.com/gWY0jPO,0,0.96,7970.5,4.78,881,1224.0,0.89,...,food,food,part of egg,"egg_yolk, yolk",n07841345,yolk%1:13:00::,yolk#1,yolk.n.01,the yellow spherical part of an egg that is su...,1
1851,zebra,zebra,https://imgur.com/xg5AAHb,0,1.00,8136.5,4.86,1066,839.0,0.87,...,animal,animal,animal,zebra,n02391049,zebra%1:05:00::,zebra#1,zebra.n.01,any of several fleet black-and-white striped A...,1
1852,zipper,zipper,https://imgur.com/T2RLBxe,0,1.00,6954.5,4.83,1478,1452.0,0.88,...,NaN,NaN,fastener,"slide_fastener, zip, zipper, zip_fastener",n04238321,zipper%1:06:00::,zipper#1,zipper.n.01,a fastener for locking together two toothed ed...,1


I discovered that my selection for the art styles though it had ample samples of art and was reasonably well categorized, it didn't provide a convenient .csv file of the dataset contents for me to be able to use immediately.  This next import will include a .csv file that I created called 'ArtStyles_DB - Sheet1.csv' with which to begin wrangling other insights from the collection as I also noted that some files were duplicates to say the least, not to mention the cleaning of ID's and other metadata information that could be relevant for the operation of the NST model down the road.

In [39]:
# Load Metadata for Artstyles to transform images by neural transfer
# params = {} 
# r.get(f"")
artstyles_df = pd.read_csv('.\Artstyles_DB\ArtStyles_DB - Sheet1.csv')
artstyles_df

,art_id,genre,art_hyperlink
0,Artstyles_DB\Surreal Symphonies (A dataset of ...,Abstract art,/kaggle/input/surreal-symphonies-a-dataset-of-...
1,Artstyles_DB\Surreal Symphonies (A dataset of ...,Abstract art,/kaggle/input/surreal-symphonies-a-dataset-of-...
2,Artstyles_DB\Surreal Symphonies (A dataset of ...,Abstract art,/kaggle/input/surreal-symphonies-a-dataset-of-...
3,Artstyles_DB\Surreal Symphonies (A dataset of ...,Abstract art,/kaggle/input/surreal-symphonies-a-dataset-of-...
4,Artstyles_DB\Surreal Symphonies (A dataset of ...,Abstract art,/kaggle/input/surreal-symphonies-a-dataset-of-...
...,...,...,...
7322,Artstyles_DB\Surreal Symphonies (A dataset of ...,Symbolism artwork,/kaggle/input/surreal-symphonies-a-dataset-of-...
7323,Artstyles_DB\Surreal Symphonies (A dataset of ...,Symbolism artwork,/kaggle/input/surreal-symphonies-a-dataset-of-...
7324,Artstyles_DB\Surreal Symphonies (A dataset of ...,Symbolism artwork,/kaggle/input/surreal-symphonies-a-dataset-of-...
7325,Artstyles_DB\Surreal Symphonies (A dataset of ...,Symbolism artwork,/kaggle/input/surreal-symphonies-a-dataset-of-...


In [59]:
# Custom tools and functions for cleaning this data set. what each function/tool does will be described prior to their declaration.

# A tool to clean the art styles id and reduce it to just the file name without the data type and it's shortened address in the system.
def art_id_rename(df, col):
    for i in df[col]:
        ilist = re.split('\\\|\.', i)
        new_art_id = ilist[4]
        df = df.replace(i, new_art_id)
    return df

# A tool to modify the hyperlink column so it can be used with the kaggle api.
def art_hyper_rename(df, col):
    dataset_home = "/cyanex1702/surreal-symphonies-a-dataset-of-diverse-art"
    for i in df[col]:
        ilist = i.split('/')
        new_art_id = str(dataset_home + ',' + ilist[-1])
        df = df.replace(i, new_art_id)
    return df


# A random integer generator used to randomly select art pieces within a genre to Neurologically transfer.
def random_select():
    return None

# A method to call the kaggle api using the artstyles 'art_hyperlink' data to collect the correct data for analysis.
def get_artstyle(df, col, genre=None, count=1, random=True):
    if type(genre) == list and random == True:
        rand = []
        selection_df = pd.DataFrame()
        for i in genre:
            subset = df.loc[df['genre'] == i, col]
            rand.append(rd.randint(0, len(subset)))
            selection_df.append(subset)
        return selection_df
    elif type(genre) == str and random == True:
        rand = 0
        subset = df.loc[df['genre'] == genre, col]
        rand = rd.randint(0, len(subset))
        user, art = df.iloc[rand, 2].split(',')
        file = k_api.dataset_download_file(user, art)
        return file

In [41]:
art_id_df = art_id_rename(artstyles_df, col='art_id')
artstyles_prep_df = art_hyper_rename(art_id_df, col='art_hyperlink')
artstyles_prep_df.iloc[400, 2]

'/cyanex1702/surreal-symphonies-a-dataset-of-diverse-art,fzdadgeo.jpg'

In [61]:
#test = get_artstyle(artstyles_prep_df,
                    #col='art_hyperlink',
                    #genre='Abstract art',
                    #count=3)
k_api.dataset_download_file("/cyanex1702/surreal-symphonies-a-dataset-of-diverse-art", 'fzdadgeo.jpg')
  

ValueError: Invalid dataset specification /cyanex1702/surreal-symphonies-a-dataset-of-diverse-art